In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from ImageWork import *
import cv2

In [62]:
def pobitovo_sravnenie_WaterMark(W1, W2 , total_bit=1024 ):
    t1 = W1==W2
    sum=t1.sum()
    return sum/total_bit*100

def psnr(W, Wr):
    e = (np.sum((W - Wr) ** 2)) / (len(W) * len(W))
    p = 10 * np.log10(255 ** 2 / e)
    return p
    

def psnrCV2(W, Wr):
    p=cv2.PSNR(W,Wr)
    return p


def one_WM(DataFrame , path_save , path_water_mark , clf):
    del DataFrame['Unnamed: 0']
    X = DataFrame.iloc[:,[1,2,3,4]].values  
    Y=DataFrame.iloc[:,[0]].values
    y_predict_image=clf.predict(X)
    y_predict_image[y_predict_image>0]=255
    y_predict_image=y_predict_image.reshape(32,32)
    img = Image.fromarray(y_predict_image.astype(np.uint8))
    img.show()
    img.save(path_save)
    W=WaterMarkLoader.load(path_water_mark)
    WR=WaterMarkLoader.load(path_save)
    print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))
    

def NO_AttackModel_Test(dataframe,path_save , title , clf, path_water_mark="Water Mark Image/WaterMarkRandom.jpg", index_image=1):
    X_dataframe = dataframe.iloc[:,[1,2,3,4]].values  
    Y_dataframe = dataframe.iloc[:,[0]].values
    X_train, X_test, y_train, y_test = train_test_split(X_dataframe, Y_dataframe, test_size=0.4, random_state=42)
    print(title)
    y_predict=clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    print(pd.DataFrame(cm))
    print()
    print(classification_report(y_test, y_predict))
    print()
    X_image_test = dataframe.iloc[:,[1,2,3,4]].values  
    i=index_image
    y=X_image_test[1024*i:1024*(i+1)]
    y_predict_image=clf.predict(y)
    y_predict_image[y_predict_image>0]=255
    y_predict_image=y_predict_image.reshape(32,32)
    img = Image.fromarray(y_predict_image.astype(np.uint8))
    img.show()
    img.save(path_save)
    W=WaterMarkLoader.load(path_water_mark)
    WR=WaterMarkLoader.load(path_save)
    print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))


In [4]:
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
del No_Attack_frame['Unnamed: 0']
X = No_Attack_frame.iloc[:,[1,2,3,4]].values  
Y=No_Attack_frame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

In [5]:
from sklearn.neural_network import MLPClassifier
clf_no_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_no_attack.fit(X_train, y_train)

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.67568280
Validation score: 0.734225
Iteration 2, loss = 0.45310213
Validation score: 0.896334
Iteration 3, loss = 0.22664440
Validation score: 0.954778
Iteration 4, loss = 0.15300323
Validation score: 0.971755
Iteration 5, loss = 0.12707220
Validation score: 0.977314
Iteration 6, loss = 0.11211234
Validation score: 0.980168
Iteration 7, loss = 0.10108586
Validation score: 0.978591
Iteration 8, loss = 0.09277103
Validation score: 0.982722
Iteration 9, loss = 0.08530878
Validation score: 0.985352
Iteration 10, loss = 0.08349619
Validation score: 0.979793
Iteration 11, loss = 0.07703186
Validation score: 0.984450
Iteration 12, loss = 0.07645089
Validation score: 0.979417
Iteration 13, loss = 0.08522400
Validation score: 0.982647
Iteration 14, loss = 0.07013130
Validation score: 0.977990
Iteration 15, loss = 0.07164427
Validation score: 0.985427
Iteration 16, loss = 0.06785500
Validation score: 0.986704
Iteration 17, loss = 0.07327704
Validation score: 0.984225
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(20, 15, 10, 5),
              max_iter=1000, verbose=True)

In [6]:
y_predict=clf_no_attack.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
print(pd.DataFrame(cm))
print(classification_report(y_test, y_predict))

       0      1
0  36802    296
1    446  34136
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     37098
           1       0.99      0.99      0.99     34582

    accuracy                           0.99     71680
   macro avg       0.99      0.99      0.99     71680
weighted avg       0.99      0.99      0.99     71680



In [7]:
X_image_test = No_Attack_frame.iloc[:,[1,2,3,4]].values  
i=4
y=X_image_test[1024*i:1024*(i+1)]
y_predict_image=clf_no_attack.predict(y)
y_predict_image[y_predict_image>0]=255
y_predict_image=y_predict_image.reshape(32,32)
img = Image.fromarray(y_predict_image.astype(np.uint8))
#img.show()
img.save("W_R/WR1.tif")
W=WaterMarkLoader.load("Water Mark Image/WaterMarkRandom.jpg")
WR=WaterMarkLoader.load("W_R/WR1.tif")
print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))

побитовое сравнение =  99.12109375


In [8]:
path_averege= "feature_vec/AverageAttack.txt"
path_histogram="feature_vec/HistogramAttack.txt"
path_GammaCor=  "feature_vec/GammaCorrection.txt"
path_JPEG50= "feature_vec/JPEG50.txt"
path_median=  "feature_vec/medianAttack.txt"
path_saltpaper="feature_vec/SaltPaperAttack.txt"
path_snarpness= "feature_vec/Sharpness.txt"

In [10]:
averege=pd.read_csv(path_averege) 
histogram=pd.read_csv(path_histogram)
GammaCor=pd.read_csv(path_GammaCor) 
JPEG50=pd.read_csv(path_JPEG50) 
median=pd.read_csv(path_median) 
saltpaper=pd.read_csv(path_saltpaper)
snarpness=pd.read_csv(path_snarpness) 
frames = [averege, histogram, GammaCor , JPEG50, median,saltpaper ,snarpness]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del saltpaper['Unnamed: 0']
del averege['Unnamed: 0']
del histogram['Unnamed: 0']
del GammaCor['Unnamed: 0']
del median['Unnamed: 0']
del JPEG50['Unnamed: 0']
del snarpness['Unnamed: 0']

DataFrame

,bit,feature_1,feature_2,feature_3,feature_4
0,1,0.0,0.5,-1.0,2.5
1,0,-6.0,-18.5,-2.5,-19.0
2,1,15.0,28.0,20.5,34.0
3,1,-33.0,-2.5,-32.0,-2.5
4,1,-12.5,-7.0,-16.5,-11.5
...,...,...,...,...,...
204795,0,-64.0,31.0,-24.5,35.0
204796,1,-14.0,-14.5,-2.5,-10.0
204797,1,12.0,6.5,-0.5,9.0
204798,1,13.0,-30.0,-22.0,-3.0


### тестирую модель обученную на noattack подсовываю атакованные изображения

In [11]:
NO_AttackModel_Test(saltpaper,"W_R/WR_SP.tif", "ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack,)

ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  38663   3721
1   1725  37811

              precision    recall  f1-score   support

           0       0.96      0.91      0.93     42384
           1       0.91      0.96      0.93     39536

    accuracy                           0.93     81920
   macro avg       0.93      0.93      0.93     81920
weighted avg       0.93      0.93      0.93     81920


побитовое сравнение =  93.06640625


In [12]:
Attack_Dota_SP = pd.read_csv('feature_vec/SaltPaperAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_SP ,"W_R/WR_SP_Dota.tif","Water Mark Image/waterMark3.jpg" , clf_no_attack )


побитовое сравнение =  94.921875


In [13]:
NO_AttackModel_Test(JPEG50,"W_R/WR_JPEG50.tif", "ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  21573  20811
1  19876  19660

              precision    recall  f1-score   support

           0       0.52      0.51      0.51     42384
           1       0.49      0.50      0.49     39536

    accuracy                           0.50     81920
   macro avg       0.50      0.50      0.50     81920
weighted avg       0.50      0.50      0.50     81920


побитовое сравнение =  52.24609375


In [14]:
Attack_Dota_jpeg = pd.read_csv('feature_vec/JPEG50Dota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_jpeg ,"W_R/WR_jpeg_Dota.tif","Water Mark Image/waterMark3.jpg" ,clf_no_attack )

побитовое сравнение =  54.6875


In [15]:
NO_AttackModel_Test(averege,"W_R/WR_averege.tif", "ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  22186  20198
1  20414  19122

              precision    recall  f1-score   support

           0       0.52      0.52      0.52     42384
           1       0.49      0.48      0.48     39536

    accuracy                           0.50     81920
   macro avg       0.50      0.50      0.50     81920
weighted avg       0.50      0.50      0.50     81920


побитовое сравнение =  50.29296875


In [17]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

побитовое сравнение =  54.39453125


In [18]:
NO_AttackModel_Test(histogram,"W_R/WR_histogram.tif", "ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  24893  17491
1   5939  33597

              precision    recall  f1-score   support

           0       0.81      0.59      0.68     42384
           1       0.66      0.85      0.74     39536

    accuracy                           0.71     81920
   macro avg       0.73      0.72      0.71     81920
weighted avg       0.74      0.71      0.71     81920


побитовое сравнение =  70.21484375


In [19]:
Attack_Dota_Histogram = pd.read_csv('feature_vec/HistogramAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Histogram ,"W_R/WR_Histogram_Dota.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

побитовое сравнение =  82.32421875


In [20]:
NO_AttackModel_Test(GammaCor,"W_R/WR_GammaCor.tif", "ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  36009   6375
1   3885  35651

              precision    recall  f1-score   support

           0       0.90      0.85      0.88     42384
           1       0.85      0.90      0.87     39536

    accuracy                           0.87     81920
   macro avg       0.88      0.88      0.87     81920
weighted avg       0.88      0.87      0.87     81920


побитовое сравнение =  90.8203125


In [21]:
Attack_Dota_GammaCorrection= pd.read_csv('feature_vec/GammaCorrectionCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_GammaCorrection ,"W_R/WR_GammaCorrection.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

побитовое сравнение =  53.515625


In [22]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  22352  20032
1  18855  20681

              precision    recall  f1-score   support

           0       0.54      0.53      0.53     42384
           1       0.51      0.52      0.52     39536

    accuracy                           0.53     81920
   macro avg       0.53      0.53      0.53     81920
weighted avg       0.53      0.53      0.53     81920


побитовое сравнение =  54.296875


In [25]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_no_attack )

побитовое сравнение =  64.16015625


In [26]:
NO_AttackModel_Test(snarpness,"W_R/WR_snarpness.tif", "ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  21931  20453
1   1557  37979

              precision    recall  f1-score   support

           0       0.93      0.52      0.67     42384
           1       0.65      0.96      0.78     39536

    accuracy                           0.73     81920
   macro avg       0.79      0.74      0.72     81920
weighted avg       0.80      0.73      0.72     81920


побитовое сравнение =  71.6796875


In [28]:
Attack_Cross_Sharpness= pd.read_csv('feature_vec/SharpnessCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_Sharpness ,"W_R/SharpnessCross.tif","Water Mark Image/waterMark2.jpg",clf_no_attack )

побитовое сравнение =  75.1953125


### тестирую модель обученную на attack image подсовываю атакованные изображения

In [29]:
X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [30]:
clf_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_attack.fit(X_train, y_train)

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.63125830
Validation score: 0.687663
Iteration 2, loss = 0.59838462
Validation score: 0.697126
Iteration 3, loss = 0.59200977
Validation score: 0.698661
Iteration 4, loss = 0.58934567
Validation score: 0.700288
Iteration 5, loss = 0.58787448
Validation score: 0.700323
Iteration 6, loss = 0.58675213
Validation score: 0.702648
Iteration 7, loss = 0.58566990
Validation score: 0.701800
Iteration 8, loss = 0.58504335
Validation score: 0.702486
Iteration 9, loss = 0.58437845
Validation score: 0.703183
Iteration 10, loss = 0.58367957
Validation score: 0.703764
Iteration 11, loss = 0.58340228
Validation score: 0.703950
Iteration 12, loss = 0.58296515
Validation score: 0.705032
Iteration 13, loss = 0.58274818
Validation score: 0.703532
Iteration 14, loss = 0.58248540
Validation score: 0.703974
Iteration 15, loss = 0.58232573
Validation score: 0.704160
Iteration 16, loss = 0.58217310
Validation score: 0.704799
Iteration 17, loss = 0.58208981
Validation score: 0.703927
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(20, 15, 10, 5),
              max_iter=1000, verbose=True)

In [31]:
y_predict=clf_attack.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
print(pd.DataFrame(cm))
print(classification_report(y_test, y_predict))

        0       1
0  223773   72156
1   97905  179606
              precision    recall  f1-score   support

           0       0.70      0.76      0.72    295929
           1       0.71      0.65      0.68    277511

    accuracy                           0.70    573440
   macro avg       0.70      0.70      0.70    573440
weighted avg       0.70      0.70      0.70    573440



In [32]:
NO_AttackModel_Test(saltpaper,"W_R/WR_SP.tif", "ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  39068   3316
1   6571  32965

              precision    recall  f1-score   support

           0       0.86      0.92      0.89     42384
           1       0.91      0.83      0.87     39536

    accuracy                           0.88     81920
   macro avg       0.88      0.88      0.88     81920
weighted avg       0.88      0.88      0.88     81920


побитовое сравнение =  86.5234375


In [34]:
Attack_Dota_SP = pd.read_csv('feature_vec/SaltPaperAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_SP ,"W_R/WR_SP_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )


побитовое сравнение =  84.27734375


In [37]:
NO_AttackModel_Test(averege,"W_R/WR_averege.tif", "ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  26705  15679
1  24552  14984

              precision    recall  f1-score   support

           0       0.52      0.63      0.57     42384
           1       0.49      0.38      0.43     39536

    accuracy                           0.51     81920
   macro avg       0.50      0.50      0.50     81920
weighted avg       0.51      0.51      0.50     81920


побитовое сравнение =  51.7578125


In [38]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )

побитовое сравнение =  51.7578125


In [39]:
NO_AttackModel_Test(histogram,"W_R/WR_histogram.tif", "ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  33186   9198
1   8942  30594

              precision    recall  f1-score   support

           0       0.79      0.78      0.79     42384
           1       0.77      0.77      0.77     39536

    accuracy                           0.78     81920
   macro avg       0.78      0.78      0.78     81920
weighted avg       0.78      0.78      0.78     81920


побитовое сравнение =  81.640625


In [40]:
Attack_Dota_Histogram = pd.read_csv('feature_vec/HistogramAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Histogram ,"W_R/WR_Histogram_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )

побитовое сравнение =  84.86328125


In [41]:
NO_AttackModel_Test(GammaCor,"W_R/WR_GammaCor.tif", "ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  40530   1854
1   7217  32319

              precision    recall  f1-score   support

           0       0.85      0.96      0.90     42384
           1       0.95      0.82      0.88     39536

    accuracy                           0.89     81920
   macro avg       0.90      0.89      0.89     81920
weighted avg       0.90      0.89      0.89     81920


побитовое сравнение =  87.3046875


In [42]:
Attack_Dota_GammaCorrection= pd.read_csv('feature_vec/GammaCorrectionCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_GammaCorrection ,"W_R/WR_GammaCorrection.tif","Water Mark Image/waterMark3.jpg",clf_attack )

побитовое сравнение =  49.21875


In [43]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  26142  16242
1  22072  17464

              precision    recall  f1-score   support

           0       0.54      0.62      0.58     42384
           1       0.52      0.44      0.48     39536

    accuracy                           0.53     81920
   macro avg       0.53      0.53      0.53     81920
weighted avg       0.53      0.53      0.53     81920


побитовое сравнение =  54.8828125


In [44]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_attack )

побитовое сравнение =  57.71484375


In [45]:
NO_AttackModel_Test(snarpness,"W_R/WR_snarpness.tif", "ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  32650   9734
1   4406  35130

              precision    recall  f1-score   support

           0       0.88      0.77      0.82     42384
           1       0.78      0.89      0.83     39536

    accuracy                           0.83     81920
   macro avg       0.83      0.83      0.83     81920
weighted avg       0.83      0.83      0.83     81920


побитовое сравнение =  83.49609375


In [46]:
Attack_Cross_Sharpness= pd.read_csv('feature_vec/SharpnessCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_Sharpness ,"W_R/SharpnessCross.tif","Water Mark Image/waterMark2.jpg",clf_attack )

побитовое сравнение =  78.3203125


In [47]:
NO_AttackModel_Test(JPEG50,"W_R/WR_JPEG50.tif", "ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  26495  15889
1  23860  15676

              precision    recall  f1-score   support

           0       0.53      0.63      0.57     42384
           1       0.50      0.40      0.44     39536

    accuracy                           0.51     81920
   macro avg       0.51      0.51      0.51     81920
weighted avg       0.51      0.51      0.51     81920


побитовое сравнение =  51.46484375


In [48]:
Attack_Dota_jpeg = pd.read_csv('feature_vec/JPEG50Dota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_jpeg ,"W_R/WR_jpeg_Dota.tif","Water Mark Image/waterMark3.jpg" ,clf_attack )

побитовое сравнение =  54.98046875


#  Обучаю сеть на одной конкретной атаке и смотрю на результат

### Медианная фильтрация

In [59]:
median=pd.read_csv(path_median) 
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
frames = [ median , No_Attack_frame ]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del median['Unnamed: 0']
DataFrame

,bit,feature_1,feature_2,feature_3,feature_4
0,1,0.0,0.0,-1.5,1.5
1,0,-3.0,-31.5,-5.0,-38.0
2,1,0.0,27.0,0.5,29.5
3,1,-31.0,-3.5,-44.0,3.5
4,1,-21.0,0.0,-18.0,-3.5
...,...,...,...,...,...
204795,0,-6.5,3.0,-1.0,3.0
204796,1,-3.5,16.5,0.5,-2.5
204797,1,0.5,-8.0,6.5,-2.5
204798,1,2.0,28.0,-2.0,49.0


In [60]:
X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

In [61]:
clf_specific_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_specific_attack.fit(X_train, y_train)

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.66260449
Validation score: 0.661546
Iteration 2, loss = 0.61370609
Validation score: 0.709923
Iteration 3, loss = 0.58499339
Validation score: 0.723182
Iteration 4, loss = 0.56657940
Validation score: 0.739183
Iteration 5, loss = 0.55635697
Validation score: 0.740723
Iteration 6, loss = 0.55101474
Validation score: 0.746357
Iteration 7, loss = 0.54729739
Validation score: 0.747521
Iteration 8, loss = 0.54526871
Validation score: 0.747671
Iteration 9, loss = 0.54396809
Validation score: 0.749061
Iteration 10, loss = 0.54360169
Validation score: 0.749023
Iteration 11, loss = 0.54020137
Validation score: 0.750601
Iteration 12, loss = 0.53939494
Validation score: 0.751390
Iteration 13, loss = 0.53872741
Validation score: 0.751164
Iteration 14, loss = 0.53835521
Validation score: 0.749099
Iteration 15, loss = 0.53758356
Validation score: 0.751540
Iteration 16, loss = 0.53705065
Validation score: 0.749437
Iteration 17, loss = 0.53655836
Validation score: 0.752479
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(20, 15, 10, 5),
              max_iter=1000, verbose=True)

In [63]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_specific_attack)

ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  22893  19491
1  19163  20373

              precision    recall  f1-score   support

           0       0.54      0.54      0.54     42384
           1       0.51      0.52      0.51     39536

    accuracy                           0.53     81920
   macro avg       0.53      0.53      0.53     81920
weighted avg       0.53      0.53      0.53     81920


побитовое сравнение =  54.98046875


In [65]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_specific_attack )

побитовое сравнение =  62.6953125


### Среднее

In [67]:
averege=pd.read_csv(path_averege) 
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
frames = [ averege , No_Attack_frame ]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del averege['Unnamed: 0']
DataFrame

X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

clf_specific_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_specific_attack.fit(X_train, y_train)

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.65988510
Validation score: 0.668607
Iteration 2, loss = 0.61363296
Validation score: 0.714581
Iteration 3, loss = 0.58360378
Validation score: 0.721867
Iteration 4, loss = 0.57001085
Validation score: 0.734488
Iteration 5, loss = 0.56315088
Validation score: 0.731596
Iteration 6, loss = 0.55905855
Validation score: 0.737868
Iteration 7, loss = 0.55591707
Validation score: 0.738469
Iteration 8, loss = 0.55397974
Validation score: 0.741136
Iteration 9, loss = 0.55246368
Validation score: 0.741136
Iteration 10, loss = 0.55051788
Validation score: 0.742638
Iteration 11, loss = 0.54933223
Validation score: 0.743990
Iteration 12, loss = 0.54846985
Validation score: 0.744967
Iteration 13, loss = 0.54803017
Validation score: 0.744704
Iteration 14, loss = 0.54768195
Validation score: 0.743014
Iteration 15, loss = 0.54679225
Validation score: 0.744892
Iteration 16, loss = 0.54647378
Validation score: 0.743051
Iteration 17, loss = 0.54644176
Validation score: 0.745530
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(20, 15, 10, 5),
              max_iter=1000, verbose=True)

In [68]:
NO_AttackModel_Test(median,"W_R/WR_average.tif", "ПРОВЕРКА ДЛЯ АТАКИ average МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_specific_attack)

ПРОВЕРКА ДЛЯ АТАКИ average МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  22750  19634
1  19105  20431

              precision    recall  f1-score   support

           0       0.54      0.54      0.54     42384
           1       0.51      0.52      0.51     39536

    accuracy                           0.53     81920
   macro avg       0.53      0.53      0.53     81920
weighted avg       0.53      0.53      0.53     81920


побитовое сравнение =  55.078125


In [69]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_specific_attack )

побитовое сравнение =  54.1015625
